EXAMPLE From

https://developer.nvidia.com/blog/speeding-up-deep-learning-inference-using-tensorflow-onnx-and-tensorrt/

The workflow consists of the following steps:

1.Convert the TensorFlow/Keras model to a .pb file.
2.Convert the .pb file to the ONNX format.
3.Create a TensorRT engine. 
4.Run inference from the TensorRT engine.


In [2]:
import tensorflow as tf

In [3]:
tf.__version__  

'2.16.2'

In [4]:
!pip install -U keras2onnx tf2onnx pycuda 

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
     ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--

  error: subprocess-exited-with-error
  
  × Building wheel for pycuda (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [155 lines of output]
      *************************************************************
      *** I have detected that you have not run configure.py.
      *************************************************************
      *** Additionally, no global config files were found.
      *** I will go ahead with the default configuration.
      *** In all likelihood, this will not work out.
      ***
      *** See README_SETUP.txt for more information.
      ***
      *** If the build does fail, just re-run configure.py with the
      *** correct arguments, and then retry. Good luck!
      *************************************************************
      *** HIT Ctrl-C NOW IF THIS IS NOT WHAT YOU WANT
      *************************************************************
      Continuing in 10 seconds...
      Continuing in 9 seconds...
      Continuing in

## Converting the model to .pb

In [11]:
import tensorflow as tf
import os

# Create models directory
os.makedirs("models", exist_ok=True)

# Load your model
model = tf.keras.models.load_model("newdata.h5")

# Method 1: Save in modern Keras format (.keras)
model.save("models/newdata_model.keras")
print("Model saved in .keras format: models/newdata_model.keras")

# Method 2: Convert to ONNX directly from the loaded model
def convert_keras_to_onnx(model, output_path):
    """
    Convert Keras model to ONNX format
    """
    try:
        import tf2onnx
        
        # Convert model to ONNX
        spec = (tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name="input"),)
        output_path_onnx = f"{output_path}.onnx"
        
        model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
        tf2onnx.utils.save_protobuf(output_path_onnx, model_proto)
        
        print(f"Model converted to ONNX: {output_path_onnx}")
        return output_path_onnx
        
    except ImportError:
        print("tf2onnx not installed. Install it with: pip install tf2onnx")
        return None
    except Exception as e:
        print(f"Error converting to ONNX: {e}")
        return None

# Method 3: Save as SavedModel (TensorFlow format)
def save_as_savedmodel(model, output_path):
    """
    Save model in SavedModel format
    """
    # For Keras 3, we don't use save_format parameter
    # Instead, we use tf.saved_model.save for explicit SavedModel format
    tf.saved_model.save(model, output_path)
    print(f"Model saved as SavedModel: {output_path}")

# Execute the conversions
print("=== Model Conversion ===")

# Save in SavedModel format
save_as_savedmodel(model, "models/newdata_savedmodel")

# Convert to ONNX
onnx_path = convert_keras_to_onnx(model, "models/newdata")

# Print model info
print(f"\nModel input shape: {model.input_shape}")
print(f"Model output shape: {model.output_shape}")
print(f"Model summary:")
model.summary()

Model saved in .keras format: models/newdata_model.keras
=== Model Conversion ===
INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets
INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets


Model saved in .keras format: models/newdata_model.keras
=== Model Conversion ===
INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets
INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets


INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets


Model saved in .keras format: models/newdata_model.keras
=== Model Conversion ===
INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets
INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets


INFO:tensorflow:Assets written to: models/newdata_savedmodel\assets


...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv4 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv4 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv4 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,31

 Total params: 139,705,443 (532.93 MB)

 Trainable params: 119,681,057 (456.55 MB)

 Non-trainable params: 20,024,384 (76.39 MB)

 Optimizer params: 2 (12.00 B)

In [10]:
onnx_path

## Converting the .pb file to ONNX 

The second step is to convert the .pb model to the ONNX format. To do this, first install tf2onnx. 

After installing tf2onnx, there are two ways of converting the model from a .pb file to the ONNX format. The first way is to use the command line and the second method is by using Python API. Run the following command:



In [7]:
# Convert to ONNX using command line (more reliable)
import subprocess
import sys
import os

def convert_to_onnx_cli(input_path, output_path, method="savedmodel"):
    """
    Convert model to ONNX using tf2onnx command line interface
    """
    try:
        if method == "savedmodel":
            # Convert from SavedModel
            cmd = [
                sys.executable, "-m", "tf2onnx.convert",
                "--saved-model", input_path,
                "--output", output_path,
                "--opset", "13"
            ]
        elif method == "keras":
            # Convert from .h5/.keras file
            cmd = [
                sys.executable, "-m", "tf2onnx.convert",
                "--keras", input_path,
                "--output", output_path,
                "--opset", "13"
            ]
        
        print(f"Running command: {' '.join(cmd)}")
        result = subprocess.run(cmd, capture_output=True, text=True, cwd=os.getcwd())
        
        if result.returncode == 0:
            print(f"✅ Successfully converted to ONNX: {output_path}")
            if result.stdout:
                print("Output:", result.stdout)
        else:
            print(f"❌ Error converting to ONNX:")
            print("STDERR:", result.stderr)
            print("STDOUT:", result.stdout)
            
    except Exception as e:
        print(f"Exception during conversion: {e}")

# Try multiple methods
print("=== Converting to ONNX ===")

# Method 1: From SavedModel
convert_to_onnx_cli("models/newdata_savedmodel", "models/newdata_from_savedmodel.onnx", "savedmodel")

# Method 2: From original H5 file
convert_to_onnx_cli("newdata.h5", "models/newdata_from_h5.onnx", "keras")

# Method 3: From new Keras file
convert_to_onnx_cli("models/newdata_model.keras", "models/newdata_from_keras.onnx", "keras")



<frozen runpy>:128: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\dsmat\AppData\Roaming\Python\Python311\site-packages\tf2onnx\convert.py", line 714, in <module>
    main()
  File "C:\Users\dsmat\AppData\Roaming\Python\Python311\site-packages\tf2onnx\convert.py", line 248, in main
    graph_def, inputs, outputs = tf_loader.from_keras(
                                 ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\dsmat\AppData\Roaming\Python\Python311\site-packages\tf2onnx\tf_loader.py", line 668, in from_keras
    _keras.backend.set_learning_phase(False)
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'keras._tf_keras.keras.backend' has no attribute 'set_learning_phase'


## Creating the TensorRT engine from ONNX
To create the TensorRT engine from the ONNX file, run the following command: 

In [ ]:
import tensorrt as trt

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
trt_runtime = trt.Runtime(TRT_LOGGER)
def build_engine(onnx_path, shape = [1,224,224,3]):

   """
   This is the function to create the TensorRT engine
   Args:
      onnx_path : Path to onnx_file. 
      shape : Shape of the input of the ONNX file. 
  """
   with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, builder.create_builder_config() as config, trt.OnnxParser(network, TRT_LOGGER) as parser:
       config.max_workspace_size = (256 << 20)
       with open(onnx_path, 'rb') as model:
           parser.parse(model.read())
       network.get_input(0).shape = shape
       engine = builder.build_engine(network, config)
       return engine

def save_engine(engine, file_name):
   buf = engine.serialize()
   with open(file_name, 'wb') as f:
       f.write(buf)
def load_engine(trt_runtime, plan_path):
   with open(plan_path, 'rb') as f:
       engine_data = f.read()
   engine = trt_runtime.deserialize_cuda_engine(engine_data)
   return engine